# `Distributor` Comparison
In `pytket-dqc` a `Distributor` transforms a pytket `Circuit` and a `NISQNetwork` into a `Distribution` describing how, and in which network modules, gates of the original circuit should be acted so as to comply with the connectivity of the network. A `Distributor` is constructed from an `Allocator`, which produces and initial `Distribution`, followed by a sequence of refinements, implemented as instances of the `Refiner` class. In this notebook we will examine some of the default distributors available through `pytket-dqc`. These have been constructed as recommended by the authors, but you may wish to develop your own.

We will use terminology that was introduced in the notebook `basic_usage`, and refer you to [Distributing circuits over heterogeneous, modular quantum computing network architectures](https://arxiv.org/abs/2305.14148) for the fullest discussion of the terminology and methods used. Throughout this notebook we will use the Chemically-Aware Ansatz circuit, used in the results of the above paper and seen below, as a running example.

In [ ]:
import json
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter

with open('chem_aware_ansatz.json', 'r') as fp:    
    circ = Circuit().from_dict(json.load(fp))

render_circuit_jupyter(circ)

We will use a particular kind of networks which we refer to as small world. Small world networks have short characteristic path lengths and have no highly connected hub nodes. In the paper [Distributing circuits over heterogeneous, modular quantum computing network architectures](https://arxiv.org/abs/2305.14148) we also perform a similar set of experiments with a scale free network, which do have highly connected hub nodes.

In [ ]:
from pytket_dqc import NISQNetwork

server_coupling = [[4, 2], [1, 2], [1, 3], [2, 3], [3, 0]]
server_qubits = {0: [0, 6], 1: [1, 9], 2: [2, 7, 8], 3: [3, 5], 4: [4, 10]}
small_world_network = NISQNetwork(
    server_coupling=server_coupling,
    server_qubits=server_qubits
)
f = small_world_network.draw_nisq_network()

The `CoverEmbedding` `Distributor` below utilises the technique introduced in [Entanglement-efficient bipartite-distributed quantum computing with entanglement-assisted packing processes](https://arxiv.org/abs/2212.12688) to distribute quantum circuits. In particular it makes use in the first instance of embedding; the merging of distributable packets either side of one or more other distributable packets to preserver entanglement.

In [ ]:
from pytket_dqc.distributors import CoverEmbedding

distribution = CoverEmbedding().distribute(circ, small_world_network, seed=0)
print("cost", distribution.cost())
print("detached gate count", distribution.detached_gate_count())
print("non local gate count", distribution.non_local_gate_count())
print("hyperedge count", len(distribution.circuit.hyperedge_list))

Under the hood, the `CoverEmbedding` `Distributor` uses the `PartitioningHeterogeneous` `Allocator` to construct an initial `Distribution`, keeps only the allocation of qubits to modules, and uses the `VertexCover` `Refiner` to implement non-local gates. However, `VertexCover`, and so `CoverEmbedding`, is designed with homogeneous networks in mind. As such there are no detached gates in the resulting `Distribution`, and each hyperedge contains gates and qubits placed in a maximum of 2 modules.

To account for this, we can act a `Refiner` after `CoverEmbedding`. The refinement passes `NeighbouringDTypeMerge` and `IntertwinedDTypeMerge` can be used to merge hyperedges so that they cover more than two modules. The resulting `Distribution` will have a reduced ebit cost due to the use of Steiner trees to distribute entanglement.

In [ ]:
from pytket_dqc.refiners import (
    NeighbouringDTypeMerge,
    IntertwinedDTypeMerge,
    SequenceRefiner,
    RepeatRefiner,
)

refiner_list = [
    NeighbouringDTypeMerge(),
    IntertwinedDTypeMerge(),
]
refiner = RepeatRefiner(SequenceRefiner(refiner_list))
refiner.refine(distribution)

print("cost", distribution.cost())
print("detached gate count", distribution.detached_gate_count())
print("non local gate count", distribution.non_local_gate_count())
print("hyperedge count", len(distribution.circuit.hyperedge_list))

The results can be further improved by making use of the `DetachedGates` `Refiner` which adjusts gate placements to make better use of detached gates. In fact the `DetachedGates` `Refiner` and the `DTypeMerge` refiners discussed above are all wrapped into a single `Distributor` provided through pytket-dqc in the form of the `CoverEmbeddingSteinerDetached` `Distributor`.

In [ ]:
from pytket_dqc.distributors import CoverEmbeddingSteinerDetached

distribution = CoverEmbeddingSteinerDetached().distribute(circ, small_world_network, seed=0)
print("cost", distribution.cost())
print("detached gate count", distribution.detached_gate_count())
print("non local gate count", distribution.non_local_gate_count())
print("hyperedge count", len(distribution.circuit.hyperedge_list))

The `PartitioningHeterogeneousEmbedding` `Distributor` is another which, like `CoverEmbeddingSteinerDetached`, makes use of embedding, detached gates, and Steiner trees. In the case of `PartitioningHeterogeneousEmbedding` the heterogeneous nature of the network is considered first and refined upon, while in the case of `CoverEmbeddingSteinerDetached` embedding is considered in the first instance. We see from the results below that  `CoverEmbeddingSteinerDetached` out performs `PartitioningHeterogeneousEmbedding` indicating that this particular circuit benefits greatly form the use of embedding, which should be considered first.

In [ ]:
from pytket_dqc.distributors import PartitioningHeterogeneousEmbedding

distribution = PartitioningHeterogeneousEmbedding().distribute(circ, small_world_network, seed=0)
print("cost", distribution.cost())
print("detached gate count", distribution.detached_gate_count())
print("non local gate count", distribution.non_local_gate_count())
print("hyperedge count", len(distribution.circuit.hyperedge_list))